<img src="https://i.imgur.com/pNKgZgL.png" alt="Fastai2 and Weights & Biases" width="400"/>

<div><img /></div>

<img src="https://i.imgur.com/uEtWSEb.png" width="650" alt="Weights & Biases" />

<div><img /></div>
<br>
<br>

## 💨 Fastai + Weights & Biases + Albumentations + pytorch-image-models

> This notebook builds upon all the other excellent notebooks show regarding this competition. In this notebook i will show how to train a `efficientnet` model from `pytorch-image-models` using the `fastai` libraray. Additionaly we will also go over how to perform image augmentations using `albumentations` and track progress using `wandb`.

### What this notebook covers ?
- model training.
- data augmentation.
- custom fastai `datablock`, `callbacks`
- track model progress using `wandb`.

### What this notebook doesn't cover?

- EDA
- creating StratifiedKFolds

In [ ]:
!git clone https://github.com/benihime91/leaf-disease-classification-kaggle
!pip install timm wandb --upgrade --quiet

In [ ]:
import sys
sys.path.append("./leaf-disease-classification-kaggle/")

In [ ]:
!wandb login a74f67fd5fae293e301ea8b6710ee0241f595a63 # [YOUR WANDB API KEY HERE]

In [ ]:
import numpy as np
import uuid 
import os
import timm
import wandb
import pandas as pd
import albumentations as A

from fastai.vision.all import *
from fastai.callback.wandb import *

from torch.distributions.beta import Beta

from src.model import _cut_model, _num_feats, _create_head

idx = uuid.uuid1()
idx = str(idx).split("-")[0]

set_seed(42)

## Create the training configuration

Specify the config file. Here we will specify all our training config parameters like : `num_classes`, `input_dims`, paths, etc etc.

In [ ]:
class Config:
    fold_num    = 0
    num_classes = 5
    csv_dir     = "./leaf-disease-classification-kaggle/data/fold_df.csv"
    image_dir   = "/kaggle/input/cassava-leaf-disease-classification/train_images/"
    input_dims  = 256
    model_arch  = "efficientnet_b3a"
    project     = "kaggle-leaf-disease-fastai-runs" # [YOUR WANDB PROJECT NAME]
    
# init config
cfg = Config()

## Prepare the dataframe with folds

We already have a stratified K-Fold data in csv format. Let's load in the the data in pandas dataframe and we will modify the dataframe for this particular task .

In [ ]:
idx2lbl = {0:"Cassava Bacterial Blight (CBB)",
          1:"Cassava Brown Streak Disease (CBSD)",
          2:"Cassava Green Mottle (CGM)",
          3:"Cassava Mosaic Disease (CMD)",
          4:"Healthy"}

In [ ]:
data             = pd.read_csv(cfg.csv_dir)
data["filePath"] = [os.path.join(cfg.image_dir, data["image_id"][idx]) for idx in range(len(data))]
data["is_valid"] = [data.kfold[n] == cfg.fold_num for n in range(len(data))]
data['label'].replace(idx2lbl, inplace=True)

# shuffle the dataset
data = data.sample(frac=1).reset_index(drop=True, inplace=False)
data.head()

## Create custom fast.ai data block

We are going to gather our data using the fast.ai `DataBlock` API.

Albumentations + fast.ai datablock, For data augmentation we are going to us the `albumentations` library.

In [ ]:
# class for albumentations transformations
# taken from : https://forums.fast.ai/t/albumentation-transformations-for-train-and-test-dataset/82642

class AlbumentationsTransform(RandTransform):
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_train_aug(): 
    return A.Compose([
        A.OneOf([
            A.RandomResizedCrop(cfg.input_dims, cfg.input_dims), 
            A.CenterCrop(cfg.input_dims, cfg.input_dims)
        ], p=0.5),
        A.Resize(cfg.input_dims, cfg.input_dims, p=1.0),
        A.HorizontalFlip(),
        A.ShiftScaleRotate(),
        A.OneOf([A.Flip(), A.Transpose(), A.IAAPerspective()]),
        A.RandomBrightnessContrast(0.1, 0.1, p=0.5),
        A.OneOf([A.CLAHE(), A.HueSaturationValue(0.2, 0.2, 0.2, p=0.5),], p=0.4),
        A.OneOf([A.CoarseDropout(), A.Cutout(), A.JpegCompression()], p=0.5),
    ])

def get_valid_aug():
    return A.Compose([A.Resize(cfg.input_dims, cfg.input_dims, p=1.0)], p=1.)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())

batch_tfms = [Normalize.from_stats(*imagenet_stats)]

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                splitter=ColSplitter(),
                get_x=lambda o: o["filePath"],
                get_y=lambda o: o["label"],
                item_tfms=item_tfms,
                batch_tfms=batch_tfms)

We can now create our `dataloaders` and have a look at our data

In [ ]:
# Load the data + preprocessing & data augmentation using the fast.ai
# datablock created above
dls = dblock.dataloaders(data)

# Let's look at a batch of data to make sure everything looks alright:
dls.show_batch(figsize=(12,12))

## Init, modify, load model for fast.ai Learner class

In this section we will load a model from the `timm` library and then modify the head of the `model` for our particular. Our custom head is based on the fast.ai custom head for used in fast.ai `cnn_learner`. Our head will be 
```
Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten()
  (2): BatchNorm1d( ... )
  (3): Dropout( ... )
  (4): Linear( ... )
  (5): ReLU( ... )
  (6): BatchNorm1d( ... )
  (7): Dropout( ... )
  (8): Linear( ... )
)
```
We will modify the `parameters` for the model for our current. For this we will use the convenience functions `_cut_model`, `_num_feats`, `_create_head`.

In [ ]:
class TransferLearningModel(Module):
    """Transfer Learning with pre-trained encoder.
    Args:
        encoder    : a pre-trained model architecture like tf_efficientnet_b3
        num_classes: total number of output classes
        lin_ftrs   : number of Linear nodes before the final output layer
    """
    def __init__(self, encoder, num_classes = cfg.num_classes, lin_ftrs=512, cut=-2):
        # remove the classifier from the encoder which are
        # the final two layers of the encoder
        self.encoder = _cut_model(encoder, cut)
        
        # calculate output features of the cut encoder
        ftrs = _num_feats(self.encoder) * 2
        
        # create the head/decoder for the encoder
        self.decoder = _create_head(ftrs, num_classes, lin_ftrs)
        apply_init(self.decoder)
        
    def forward(self, xb):
        # 1. Feature extraction:
        feats = self.encoder(xb)
        
        # 2. Decoder (returns logits):
        logits = self.decoder(feats)
        return logits

We will now create custom `splitter` to pass along to `Learner` to take advantage of transfer learning. In order to use transfer learning efficiently, we will want to freeze the pretrained model at first, and train only the head. The `params` function is useful to return all parameters of the model, so we can create a simple splitter like so:

In [ ]:
def splitter(net): 
    """
    returns parameters of the classifer and the base of the model
    required for gradual freezing/unfrezing and discriminative
    lr techniques for fast.ai
    """
    return [params(net.encoder), params(net.decoder)]

## Init fast.ai Learner and start training

Let's now create our `Learner` object.

Implement cutmix callback from [here](https://github.com/fastai/fastai/blob/master/fastai/callback/cutmix.py#L10) .

In [ ]:
# straight copy from : https://docs.fast.ai/callback.cutmix#CutMix
class CutMix(Callback):
    "Implementation of `https://arxiv.org/abs/1905.04899`"
    run_after,run_valid = [Normalize],False
    def __init__(self, alpha=1.): self.distrib = Beta(tensor(alpha), tensor(alpha))
    def before_fit(self):
        self.stack_y = getattr(self.learn.loss_func, 'y_int', False)
        if self.stack_y: self.old_lf,self.learn.loss_func = self.learn.loss_func,self.lf

    def after_fit(self):
        if self.stack_y: self.learn.loss_func = self.old_lf

    def before_batch(self):
        W, H = self.xb[0].size(3), self.xb[0].size(2)
        lam = self.distrib.sample((1,)).squeeze().to(self.x.device)
        lam = torch.stack([lam, 1-lam])
        self.lam = lam.max()
        shuffle = torch.randperm(self.y.size(0)).to(self.x.device)
        xb1,self.yb1 = tuple(L(self.xb).itemgot(shuffle)),tuple(L(self.yb).itemgot(shuffle))
        nx_dims = len(self.x.size())
        x1, y1, x2, y2 = self.rand_bbox(W, H, self.lam)
        self.learn.xb[0][:, :, x1:x2, y1:y2] = xb1[0][:, :, x1:x2, y1:y2]
        self.lam = (1 - ((x2-x1)*(y2-y1))/float(W*H)).item()

        if not self.stack_y:
            ny_dims = len(self.y.size())
            self.learn.yb = tuple(L(self.yb1,self.yb).map_zip(torch.lerp,weight=unsqueeze(self.lam, n=ny_dims-1)))

    def lf(self, pred, *yb):
        if not self.training: return self.old_lf(pred, *yb)
        with NoneReduce(self.old_lf) as lf:
            loss = torch.lerp(lf(pred,*self.yb1), lf(pred,*yb), self.lam)
        return reduce_loss(loss, getattr(self.old_lf, 'reduction', 'mean'))

    def rand_bbox(self, W, H, lam):
        cut_rat = torch.sqrt(1. - lam)
        cut_w = (W * cut_rat).type(torch.long)
        cut_h = (H * cut_rat).type(torch.long)
        # uniform
        cx = torch.randint(0, W, (1,)).to(self.x.device)
        cy = torch.randint(0, H, (1,)).to(self.x.device)
        x1 = torch.clamp(cx - cut_w // 2, 0, W)
        y1 = torch.clamp(cy - cut_h // 2, 0, H)
        x2 = torch.clamp(cx + cut_w // 2, 0, W)
        y2 = torch.clamp(cy + cut_h // 2, 0, H)
        return x1, y1, x2, y2

In [ ]:
encoder = timm.create_model(cfg.model_arch, pretrained=True, num_classes=dls.c)

model   = TransferLearningModel(encoder, cut=-4)

In [ ]:
model

In [ ]:
run_name = f"{cfg.model_arch}-fold={cfg.fold_num}-{idx}"
wandb.init(project=cfg.project, name=run_name)

In [ ]:
loss  = CrossEntropyLossFlat()
learn = (Learner(dls, model, loss_func=loss, splitter=splitter, metrics=accuracy, cbs=[WandbCallback(), CutMix()]).to_native_fp16())

We are now provided with a Learner object. We will first freeze the model i.e., only the weights of the head can be updated. 
In order to train a model, we need to find the most optimal learning rate, which can be done with fastai's learning rate finder:

In [ ]:
learn.freeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(22, 1e-03)

>Tip: Save model weights after each each run to note lose progress.

In [ ]:
torch.save(learn.model.state_dict(), "stage-1.pt")

In [ ]:
learn.model.load_state_dict(torch.load("stage-1.pt"))

## Unfreeze the whole model and train

In [ ]:
# train all the parameters of the model
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(12, slice(1e-06, 1e-04))

We can now `save` the weights of our PyTorch model or export the fastai `Learner`. I prefere saving the weights of the model so than i can simple use one the inference notebooks already uploaded to make inference

In [ ]:
save_dir = f"/kaggle/working/{cfg.model_arch}-{cfg.input_dims}-fold={cfg.fold_num}.pt"
torch.save(learn.model.state_dict(), save_dir)

> Optional: Update saved weights file to `wandb` and finish the `wandb` `run`.

In [ ]:
wandb.save(save_dir)
wandb.finish()

## References

- https://www.kaggle.com/tanlikesmath/cassava-classification-eda-fastai-starter
- https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-train-amp-aug
- https://www.kaggle.com/muellerzr/cassava-fastai-starter
- https://www.kaggle.com/muellerzr/recreating-abhishek-s-tez-with-fastai